#### №VI.9.32 ####

| Год          | Население      |
| ------------ | -------------- |
|     1910     |    92 228 496  |
|     1920     |   106 021 537  |
|     1930     |   123 202 624  |
|     1940     |   132 164 569  |
|     1950     |   151 325 798  |
|     1960     |   179 323 175  |
|     1970     |   203 211 926  |
|     1980     |   226 545 805  |
|     1990     |   248 709 873  |
|     2000     |   281 421 906  |

In [73]:
import numpy as np
import scipy
import bisect

num_of_people = np.array([92228496, 106021537, 123202624, 132164569, 151325798, 179323175, 203211926, 226545805, 248709873, 281421906])
year = np.array([1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000])

Требуется вычислить экстраполированное значение численности населения в 2010 году и сравнить его с истинным значением, равным 308 745 538.

a) Построим интерполянт в форме Ньютона.
Полином Ньютона: $$ N(x) = b_0 + b_{1}(x - x_0) + b_{2}(x - x_0)(x - x_1) + ... + b_n(x-x_0)(x-x_1)...(x-x_{n - 1}) $$

In [74]:
def b_coef_finder(year, num_of_people):
    size = len(year)
    sp_tbl = np.zeros([size, size])
    for i in range (0, size):
        sp_tbl[i, 0] = num_of_people[i]
    for j in range (1, size):
        for i in range (1, size):
            sp_tbl[i, j] = (sp_tbl[i, j - 1] - sp_tbl[i - 1, j - 1]) / (year[i] - year[i - j])
    res = np.zeros([size])
    for i in range(0, size):
        res[i] = sp_tbl[i, i]
    return res

def N_polynomial(year, num_of_people, x):
    sp_tbl = b_coef_finder(year, num_of_people)
    size = len(sp_tbl)
    temp = 1
    res = sp_tbl[0]
    for i in range (size - 1):
        temp = temp * (x - year[i])
        res = res + sp_tbl[i + 1] * temp
    return res

Для 2010 года:

In [75]:
real_result = 308745538
interpol_result = N_polynomial(year, num_of_people, 2010.0)

print("Истинное значение:", real_result)
print("Полученное значение:", int(interpol_result))

Истинное значение: 308745538
Полученное значение: 827906509


б) Построим сплайн-аппроксимацию. Ищем сплайн в виде:
$$ S_k = a_k + \frac{b_k}{1!}(x - x_k) + \frac{c_k}{2!}(x - x_k)^2 + \frac{d_k}{3!}(x - x_k)^3 $$

В ячейке ниже представлен код для нахождения сплайна.

In [76]:
def diff_adj(x):
    size = len(x) - 1
    res = np.zeros(size)
    for i in range(size):
        res[i] = x[i + 1] - x[i]
    return res

def target_count(n, h, y):
    res_arr = np.zeros(n)
    for i in range (1, n-1):
        res_arr[i] = 6 * ((y[i + 1] - y[i]) / h[i] - (y[i] - y[i - 1]) / h[i - 1]) / (h[i] + h[i - 1])
    return res_arr

def tridiag_matr(n, h):
    A = np.zeros(n - 1)
    C = np.zeros(n - 1)
    for i in range (n - 2):
        A[i] = h[i] / (h[i] + h[i + 1])
        C[i + 1] = h[i] / (h[i] + h[i + 1])
    B = np.zeros(n)
    for i in range (n):
        B[i] = 2
    return A, B, C

def calc_tridiag(A, B, C, D):
    c_p = np.append(C, 0)
    d_p = np.zeros(len(B))
    result = np.zeros(len(B))
    for i in range(0, len(B)):
        if i == 0:
            c_p[i] = C[i] / B[i]
            d_p[i] = D[i] / B[i]
        else:
            c_p[i] = c_p[i] / (B[i] - c_p[i - 1] * A[i - 1])
            d_p[i] = (D[i] - d_p[i - 1] * A[i - 1]) / (B[i] - c_p[i - 1] * A[i - 1])
    result[-1] = d_p[-1]
    for i in range(len(B) - 2, -1, -1):
        result[i] = d_p[i] - c_p[i] * result[i + 1]
    return result

def spline_get_res(x, y, year):
    n = len(x)
    h = diff_adj(x)
    A, B, C = tridiag_matr(n, h)
    D = target_count(n, h, y)
    tridiag_solved = calc_tridiag(A, B, C, D)
    tmp = np.array([[(tridiag_solved[i + 1] - tridiag_solved[i]) * h[i] * h[i] / 6, tridiag_solved[i] * h[i] * h[i] / 2, (y[i + 1] - y[i] - (tridiag_solved[i + 1]+  2 * tridiag_solved[i]) * h[i] * h[i] / 6), y[i]] for i in range(n - 1)])
    ind = min(bisect.bisect(x, year) - 1, n - 2)
    z = (year - x[ind]) / h[ind]
    C = tmp[ind]
    return (((C[0] * z) + C[1]) * z + C[2]) * z + C[3]

Для 2010 года:

In [77]:
spline_result = spline_get_res(year, num_of_people, 2010.0)

print("Истинное значение:", real_result)
print("Полученное значение:", int(spline_result))

Истинное значение: 308745538
Полученное значение: 314133939


Подсчитаем погрешности для методов, предложенных в пунктах "а" и "б":

In [78]:
err_1 = int(np.abs((interpol_result / real_result) - 1) * 100)
err_2 = int(np.abs((spline_result / real_result) - 1) * 100)

print("Погрешности в процентах для обоих методов:")
print("Интерполянт в форме Ньютона:", err_1, "%")
print("Сплайн-аппроксимация:", err_2, "%")

Погрешности в процентах для обоих методов:
Интерполянт в форме Ньютона: 168 %
Сплайн-аппроксимация: 1 %


Тем самым, мы получаем намного более точный результат при использовании сплайн-аппроксимации, чем при использовании интерполянта в форме Ньютона.